In [9]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd



In [2]:
rawdata_path = 'rawdata/'
data_path = 'data/'

In [16]:
# 인허가 정보 파일 확인
file_list = os.listdir(f'{rawdata_path}/서울시 1인가구/')
file_list

['2022.10월_10개 관심집단 수.xlsx',
 '2022.11월_10개 관심집단 수.xlsx',
 '2022.12월_10개 관심집단 수.xlsx',
 '2022.1월_10개 관심집단수.xlsx',
 '2022.2월_10개 관심집단수.xlsx',
 '2022.3월_10개 관심집단수.xlsx',
 '2022.4월_10개 관심집단수.xlsx',
 '2022.5월_10개 관심집단수.xlsx',
 '2022.6월_10개 관심집단수.xlsx',
 '2022.7월_10개 관심집단수.xlsx',
 '2022.8월_10개 관심집단수.xlsx',
 '2022.9월_10개 관심집단수.xlsx',
 '2023.10월_10개 관심집단 수.xlsx',
 '2023.11월_10개 관심집단 수.xlsx',
 '2023.12월_10개 관심집단 수.xlsx',
 '2023.1월_10개 관심집단 수.xlsx',
 '2023.2월_10개 관심집단 수.xlsx',
 '2023.3월_10개 관심집단 수.xlsx',
 '2023.4월_10개 관심집단 수.xlsx',
 '2023.5월_10개 관심집단 수.xlsx',
 '2023.6월_10개 관심집단 수.xlsx',
 '2023.7월_10개 관심집단 수.xlsx',
 '2023.8월_10개 관심집단 수.xlsx',
 '2023.9월_10개 관심집단 수.xlsx',
 '2024.1월_10개 관심집단 수.xlsx',
 '2024.2월_10개 관심집단 수.xlsx',
 'zip']

In [17]:
file_list = file_list[:-1]
file_list

['2022.10월_10개 관심집단 수.xlsx',
 '2022.11월_10개 관심집단 수.xlsx',
 '2022.12월_10개 관심집단 수.xlsx',
 '2022.1월_10개 관심집단수.xlsx',
 '2022.2월_10개 관심집단수.xlsx',
 '2022.3월_10개 관심집단수.xlsx',
 '2022.4월_10개 관심집단수.xlsx',
 '2022.5월_10개 관심집단수.xlsx',
 '2022.6월_10개 관심집단수.xlsx',
 '2022.7월_10개 관심집단수.xlsx',
 '2022.8월_10개 관심집단수.xlsx',
 '2022.9월_10개 관심집단수.xlsx',
 '2023.10월_10개 관심집단 수.xlsx',
 '2023.11월_10개 관심집단 수.xlsx',
 '2023.12월_10개 관심집단 수.xlsx',
 '2023.1월_10개 관심집단 수.xlsx',
 '2023.2월_10개 관심집단 수.xlsx',
 '2023.3월_10개 관심집단 수.xlsx',
 '2023.4월_10개 관심집단 수.xlsx',
 '2023.5월_10개 관심집단 수.xlsx',
 '2023.6월_10개 관심집단 수.xlsx',
 '2023.7월_10개 관심집단 수.xlsx',
 '2023.8월_10개 관심집단 수.xlsx',
 '2023.9월_10개 관심집단 수.xlsx',
 '2024.1월_10개 관심집단 수.xlsx',
 '2024.2월_10개 관심집단 수.xlsx']

In [71]:
group1 = ['total_all', 'solo_all']
group2 = [x for x in range(20, 80, 5)]

age_groups = group1 + group2

solo = pd.DataFrame()

for file_name in file_list:

    file_date = re.sub(r'\D', '', file_name)[:-2]

    folder_path = f'{rawdata_path}/서울시 1인가구/'
    path = folder_path + file_name

    temp = pd.read_excel(path)

    temp = temp.iloc[:, :7]

    ## 1인 가구수, 총인구수 소수점 단위 반올림
    temp['1인가구수'] = temp['1인가구수'].apply(lambda x: round(x))
    temp['총인구'] = temp['총인구'].apply(lambda x: round(x))  


    temp_all = temp.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
    temp_all.columns = [f'total_all_{file_date}', f'solo_all_{file_date}']

    age_groups = [x for x in range(20, 80, 5)]

    for age in age_groups:
        temp_age = temp[temp['연령대'].isin([age])]
        temp_group = temp_age.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
        temp_group.columns = [f'total_{age}_{file_date}', f'solo_{age}_{file_date}']
        temp_all = pd.concat([temp_all, temp_group], axis = 1)

    temp_all.reset_index(inplace = True, drop = False)

    

    globals()[f'solo{file_date}'] = temp_all.merge(temp[['행정동코드', '자치구', '행정동명']],
                    how = 'left',
                    on = '행정동코드')

    globals()[f'solo{file_date}']['행정동명'] = globals()[f'solo{file_date}']['행정동명'].apply(lambda x: re.sub(r'\·', '', x))

    globals()[f'solo{file_date}'].drop_duplicates(inplace = True)
    globals()[f'solo{file_date}'].reset_index(drop = True, inplace = True)
    globals()[f'solo{file_date}'].to_csv(f'{data_path}/solo/solo{file_date}.csv', index = False)

    solo = pd.concat([solo, globals()[f'solo{file_date}']])

In [72]:
solo

,행정동코드,total_all_202210,solo_all_202210,total_20_202210,solo_20_202210,total_25_202210,solo_25_202210,total_30_202210,solo_30_202210,total_35_202210,...,total_55_20242,solo_55_20242,total_60_20242,solo_60_20242,total_65_20242,solo_65_20242,total_70_20242,solo_70_20242,total_75_20242,solo_75_20242
0,1101053,7842.0,1806.0,582.0,192.0,687.0,234.0,781.0,209.0,754.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1101054,2403.0,824.0,176.0,87.0,172.0,89.0,193.0,89.0,211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1101055,9084.0,1557.0,1147.0,215.0,837.0,150.0,703.0,181.0,652.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1101056,15344.0,1626.0,1260.0,134.0,1166.0,127.0,1172.0,140.0,1075.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1101057,6337.0,801.0,453.0,49.0,423.0,49.0,455.0,64.0,555.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,1125070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1102.0,572.0,841.0,451.0,728.0,95.0,419.0,50.0,361.0,103.0
420,1125071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2518.0,260.0,2045.0,224.0,1508.0,252.0,1040.0,174.0,1496.0,566.0
421,1125072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3044.0,461.0,2568.0,425.0,1997.0,404.0,1397.0,287.0,1937.0,743.0
422,1125073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3303.0,611.0,2717.0,522.0,2188.0,516.0,1516.0,363.0,2308.0,829.0
